In [ ]:
import pandas as pd
from analysis import *
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import binom_test

In [ ]:
surge_annotations = data.surge_evaluation.annotation_dataframe()
surge_annotations_comparative = data.surge_evaluation.comparative_annotation_dataframe()

surge_annotations

# 8 Comprehensive Analysis

### Metric Sensitivity

In [ ]:
from itertools import combinations
from scipy.stats import ttest_ind

def p_vals(df: pd.DataFrame, test='t', downsample=None):
    """
    :param df: (bot, data point) x 1 -> score
    :param test: statistical test function (t for t test, p for prop test, s for sign test)
    :param downsample: number of samples ber bot to subsample without replacement for the analysis
    :return: p values of test on each bot pair (pd.Series)
    """
    seed = 123
    bots = set(df.index.get_level_values(0))
    num_bots = len(bots)
    bot_pairs = list(combinations(bots, 2))
    result = {}
    for ba, bb in bot_pairs:
        if test == 't':
            if downsample:
                a = df.xs(ba).sample(downsample, random_state=seed).to_numpy().squeeze()
                b = df.xs(bb).sample(downsample, random_state=seed).to_numpy().squeeze()
            else:
                a = df.xs(ba).to_numpy().squeeze()
                b = df.xs(bb).to_numpy().squeeze()
            t, p = ttest_ind(a, b, equal_var=False)
        elif test == 'p':
            if downsample:
                a = df.xs(ba).sample(downsample, random_state=seed).to_numpy().squeeze()
                b = df.xs(bb).sample(downsample, random_state=seed).to_numpy().squeeze()
            else:
                a = df.xs(ba).to_numpy().squeeze()
                b = df.xs(bb).to_numpy().squeeze()
            z, p = proportions_ztest(count=[
                sum(a), sum(b)
            ], nobs=[
                len(a), len(b)
            ])
        elif test == 's':
            # sign test
            comp_data = df.xs((ba, bb), level=[sym.bot, sym.bot_cmp])
            if downsample:
                comp_data = comp_data.sample(downsample, random_state=seed)
            a = comp_data.to_numpy().squeeze() == 1
            b = comp_data.to_numpy().squeeze() == -1
            p = binom_test(sum(a), sum(a)+sum(b), p=0.5)
        else:
            raise ValueError('invalid arg for param "test"')
        result[(ba, bb)] = p
    result_series = pd.Series(result.values(), result)
    return result_series

@to_file
def p_values_comparing_bots(evaluation, downsample=None):
    comp_annotations = get_singly_annotated(evaluation.comparative_annotation_dataframe(), seed=123)
    annotations = get_singly_annotated(evaluation.annotation_dataframe(), seed=123)
    prop_annotations = annotations.xs(
        category.behavior, level=sym.category, drop_level=False
    )
    mean_annotations = annotations.drop(
        index=category.behavior, level=sym.category
    ).drop(
        index=category.comparative, level=sym.category
    )
    mean_ps = mean_annotations.groupby(
        [sym.category, sym.label]
    ).apply(lambda x: p_vals(x, test='t', downsample=downsample))
    prop_ps = prop_annotations.groupby(
        [sym.category, sym.label]
    ).apply(lambda x: p_vals(x, test='p', downsample=downsample))
    comp_groups = comp_annotations.groupby(sym.label)
    comp_ps = comp_groups.apply(lambda x: p_vals(x, test='s', downsample=downsample))
    comp_ps = pd.concat({category.comparative: comp_ps}, names=[sym.category])
    result = pd.concat([prop_ps, mean_ps, comp_ps], axis=0)
    return result

p_values_comparing_bots(data.surge_evaluation, downsample=32, load='results/p_values_comparing_bots_downsampled').round(4)

In [ ]:
p_values_comparing_bots(data.surge_evaluation, load='results/t_test_p_values_comparing_bots').round(4)

### Predictive Validity

In [ ]:
from statsmodels.miscmodels.ordinal_model import OrderedModel
from statsmodels.regression.linear_model import OLS as LinearModel
from statsmodels.discrete.discrete_model import Logit as LogisticModel
from statsmodels.tools.tools import add_constant


def all_dialogue_metrics(data):
    static: pd.DataFrame = data.surge_evaluation.annotation_dataframe()
    static = get_singly_annotated(static, seed=123)
    reindexed = static.reset_index()
    items = reindexed[sym.item]
    dialogues = [e[0] if isinstance(e, tuple) else e for e in items]
    reindexed['dialogue'] = dialogues
    reindexed.set_index(
        [sym.bot, sym.category, sym.label, 'dialogue', sym.item],
        inplace=True, verify_integrity=True
    )
    ld = reindexed.xs(category.likert_dialogue, level=sym.category)
    ld = ld.droplevel(sym.bot).droplevel(sym.item)
    ld.columns = ['score']
    ldq = ld.xs(scale.quality, level=sym.label)
    ldq.columns = ['quality']

    lt = reindexed.xs(category.likert_turn, level=sym.category)
    lt = lt.groupby([sym.label, 'dialogue']).mean()
    lt.columns = ['score']
    ltq = lt.xs(scale.quality, level=sym.label)
    ltq.columns = ['quality']

    be = reindexed.xs(category.behavior, level=sym.category)
    be = be.groupby([sym.label, 'dialogue']).mean()
    be.columns = ['score']

    interactive = data.dialogue_collection.annotation_dataframe()
    idq = interactive.xs((category.likert_dialogue, scale.quality), level=(sym.category, sym.label))
    idq = idq.droplevel(0)

    ds = pd.concat(
        [lt, be, ld],
        keys=[category.likert_turn, category.behavior, category.likert_dialogue],
        names=[sym.category, sym.label, 'dialogue']
    )
    likert_dialogue_quality_features = ds.join(ldq, on='dialogue')
    likert_turn_quality_features = ds.join(ltq, on='dialogue')
    interactive_dialogue_quality_features = ds.join(idq, on='dialogue')
    interactive_dialogue_quality_features.columns = ['score', 'quality']

    interactive_comparisons = data.dialogue_collection.comparative_annotation_dataframe()
    surge_comparisons = get_singly_annotated(data.surge_evaluation.comparative_annotation_dataframe(), seed=123)
    compared_dialogues = surge_comparisons.index.get_level_values('dialogues')
    unique_compared_dialogues = {tuple(x) for x in {frozenset(y) for y in compared_dialogues}}
    comparison_map = dict(unique_compared_dialogues)
    compared_selector = [
        pair in unique_compared_dialogues
        for pair in interactive_comparisons.index.get_level_values('dialogues')
    ]
    comparative: pd.DataFrame = interactive_comparisons.loc[compared_selector, :]
    compared_selector = [
        pair in unique_compared_dialogues
        for pair in surge_comparisons.index.get_level_values('dialogues')
    ]
    surge_comparisons: pd.DataFrame = surge_comparisons.loc[compared_selector, :]
    comparative_quality = comparative.xs(scale.quality, level=sym.label)
    comparative_quality.index = [first for _, _, (first, second) in comparative_quality.index.values]
    comparative_quality.columns = ['quality']
    surge_comparisons.index = pd.MultiIndex.from_arrays(
        list(zip(*[
            (category.comparative, label, left)
            for _, _, label, (left, right) in surge_comparisons.index.values
        ])),
        names=[sym.category, sym.label, 'dialogue']
    )
    surge_comparisons.columns = ['score']
    filtered_ds = ds.loc[[(c, l, d) for c, l, d in ds.index.values if d in comparison_map]]
    compared_features = ds.loc[[(c, l, comparison_map[d]) for c, l, d in filtered_ds.index.values]]
    comparative_features = filtered_ds.to_numpy() - compared_features.to_numpy()
    filtered_ds['diff'] = comparative_features.squeeze().tolist()
    del filtered_ds['score']
    filtered_ds.columns = ['score']
    filtered_ds = pd.concat([filtered_ds, surge_comparisons], axis=0)
    comparative_quality_features = filtered_ds.join(comparative_quality, on='dialogue')
    icq = comparative_quality_features
    icq = icq[icq['quality'] != 0]
    icq.loc[:,'quality'] = (icq['quality'] > 0).astype(int)

    return (
        likert_dialogue_quality_features,
        likert_turn_quality_features,
        icq,
        interactive_dialogue_quality_features
    )

all_dialogue_metrics(data)

def regressions(df, quality_column_name=None, model='linear'):
    """
    :param df: dialogue x (*features, quality) -> value
    :return: *(coef, low, high), mcfadden r^2
    """
    if not quality_column_name:
        quality_column_name = df.columns[-1]
    qualities = df[quality_column_name]
    features = [f for f in df.columns if f != quality_column_name]
    if model == 'ordinal':
        model = OrderedModel(qualities, df[features], distr='logit')
        results = model.fit()
        coefs = {f: results.params[f] for f in features}
        prsqrd = results.prsquared
        result = {stat.mcfad_r2: prsqrd, stat.p_of_llr_test: results.llr_pvalue}
    elif model == 'linear':
        x = add_constant(df[features])
        y = qualities
        model = LinearModel(y, x)
        results = model.fit()
        coefs = {f: results.params[f] for f in features}
        rsquared = results.rsquared
        result = {**coefs, stat.r2: rsquared, stat.p_of_f_test: results.f_pvalue}
    elif model == 'logistic':
        x = add_constant(df[features])
        y = qualities.astype(bool)
        model = LogisticModel(y, x)
        results = model.fit()
        coefs = {f: results.params[f] for f in features}
        prsqrd = results.prsquared
        result = {**coefs, stat.mcfad_r2: prsqrd, stat.p_of_llr_test: results.llr_pvalue}
    else:
        raise ValueError('Param "model" must be one of {"linear", "ordinal", "logistic"}')
    return pd.Series(result.values(), result)



In [ ]:
@to_file
def dialogue_quality_regressions(data):
    ldq, ltq, icq, idq = all_dialogue_metrics(data)
    ldq_groups = ldq.groupby([sym.category, sym.label])
    ltq_groups = ltq.groupby([sym.category, sym.label])
    icq_groups = icq.groupby([sym.category, sym.label])
    idq_groups = idq.groupby([sym.category, sym.label])
    names = ['Predicted', 'Metric']
    linear_compare_result = icq_groups.apply(lambda x: regressions(x, model='logistic'))
    linear_compare_result.columns = pd.MultiIndex.from_arrays(
        [['Interactive Comparison']*3,
        ['LC Coefficient', 'LC Pseudo R-Squared', stat.p_of_llr_test]],
        names=names
    )
    linear_result = ldq_groups.apply(lambda x: regressions(x, model='linear'))
    linear_result.columns = pd.MultiIndex.from_arrays(
        [['Likert Dialogue Quality']*3,
        ['LR Coefficient', 'LR R-Squared', stat.p_of_f_test]],
        names=names
    )
    ordinal_result = ldq_groups.apply(lambda x: regressions(x, model='ordinal'))
    ordinal_result.columns = pd.MultiIndex.from_arrays(
        [['Likert Dialogue Quality']*2,
        ['OR Pseudo R-Squared', stat.p_of_llr_test]],
        names=names
    )
    linear_turn_result = ltq_groups.apply(regressions)
    linear_turn_result.columns = pd.MultiIndex.from_arrays(
        [['Likert Turn Quality']*3,
        ['LR Coefficient', 'LR R-Squared', stat.p_of_f_test]],
        names=names
    )
    interactive_dial_result = idq_groups.apply(regressions)
    interactive_dial_result.columns = pd.MultiIndex.from_arrays(
        [['Interactive Likert']*3,
        ['LR Coefficient', 'LR R-Squared', stat.p_of_f_test]],
        names=names
    )
    result = pd.concat(( linear_compare_result, interactive_dial_result, linear_result, linear_turn_result), axis=1)
    return result.round(5)

regs = dialogue_quality_regressions(
    data,
    load='results/dialogue_quality_regressions'
)
regs

In [ ]:
to_plot_regs = regs[[("Likert Dialogue Quality", "LR R-Squared"), ("Likert Dialogue Quality", "P value of F-test")]]
to_plot_regs = to_plot_regs.drop(("likert dialogue", "quality"))
to_plot_regs = to_plot_regs.reset_index()
to_plot_regs

In [ ]:
regs = prettify(regs, to_csv="results/paper/predictive_validity.csv")

# Incremental Validity

In [ ]:
def drop_column_level_duplication(df: pd.DataFrame, columns, levels=None):
    if levels is None:
        levels = list(range(len(columns)))
    level_columns = df.xs(columns, axis=1, level=levels)
    unique = level_columns.iloc[:,0].to_frame()
    unique.columns = [columns]
    dropped = df.drop(columns=columns, level=levels)
    result = pd.concat([dropped, unique], axis=1)
    return result

def multivariate_regression(df: pd.DataFrame, model='linear'):
    def apply_regressions(df: pd.DataFrame):
        unstacked = df.unstack([sym.category, sym.label])
        unstacked = drop_column_level_duplication(unstacked, 'quality', 0)
        results = regressions(unstacked, quality_column_name='quality', model=model)
        return results
    result = apply_regressions(df)
    result.index = [
        (idx[1] if isinstance(idx, tuple) else idx)
        for idx in result.index.values
    ]
    return result.round(5)

from collections import namedtuple

@to_file
def incremental_regression(
        df: pd.DataFrame,
        categories,
        model='linear',
        beam=1,
        select='backward',
        exclusions=(),
):
    data_points = set(df.index.get_level_values('dialogue'))
    num_data_points = len(data_points)
    adjust = lambda r2, f: 1 - (1 - r2) * ((num_data_points - 1) / (num_data_points - f))
    Step: type = namedtuple('Step', ('r2', 'p', 'feature'))
    class Path(list):
        def metric(self):
            # if len(self) == 0: return 0
            # else: return self[-1].llr if len(self) == 1 else self[-1].llr / self[-2].llr
            return self.r2
        @property
        def r2(self):
            return (adjust(self[-1].r2, len(self)) if model == 'linear' else self[-1].r2) if self else 0
        # @property
        # def adj_r2(self):
        #     return adjust(self.r2, len(self))
        @property
        def p(self): return self[-1].p if self else 1
        @property
        def features(self): return {x.feature for x in self}
    r2_name = stat.r2 if model=='linear' else stat.mcfad_r2
    p_name = stat.p_of_f_test if model=='linear' else stat.p_of_llr_test
    frontier = [Path()]
    feature_pool = {
        x[:2] for x in df.index.values
        if (not (x in exclusions or x[1] in exclusions))
        and x[0] in categories
    }
    for _ in feature_pool:
        new_frontier = []
        for path in frontier:
            for candidate in feature_pool - path.features:
                if select == 'forward':
                    candidate_features = path.features | {candidate}
                elif select == 'backward':
                    candidate_features = feature_pool - path.features
                else:
                    raise ValueError('param select must be one of {"forward", "backward"}')
                row_mask = [
                    x[:2] in candidate_features
                    and (not (x in exclusions or x[1] in exclusions))
                    and x[0] in categories
                    for x in df.index.values
                ]
                candidate_df = df.loc[row_mask, :]
                candidate_results = multivariate_regression(candidate_df, model=model)
                r2 = candidate_results[r2_name].item()
                p = candidate_results[p_name]
                new_frontier.append(Path([*path, Step(r2, p, candidate)]))
        frontier = sorted(new_frontier, key=lambda x: x.metric(), reverse=True)[:beam]
    result = {step.feature: {r2_name: step.r2, p_name: step.p} for step in frontier[0]}
    return pd.DataFrame(result.values(), result)


ldq, ltq, icq, idq = all_dialogue_metrics(data)
regs = incremental_regression(
    ldq, (category.likert_turn, category.behavior), beam=1, exclusions=[scale.quality],
    load='results/dialogue_incremental_regressions'
)
regs

In [ ]:
behavior_regs = incremental_regression(
    ldq, (category.behavior,), beam=10,
    load='results/behavior_incremental_regressions'
)
behavior_regs

In [ ]:
behavior_regs = incremental_regression(
    idq, (category.behavior,), beam=10,
    load='results/behavior_incremental_regressions_interactive'
)
behavior_regs

In [ ]:
behavior_regs_comp = incremental_regression(
    icq, (category.behavior,), beam=10, model='logistic',
    load='results/behavior_incremental_regressions_comparative'
)
behavior_regs_comp

In [ ]:
likert_turn_regs = incremental_regression(
    ldq, (category.likert_turn,), beam=10, exclusions=[scale.quality],
    load='results/likert_turn_incremental_regressions'
)
likert_turn_regs

In [ ]:
likert_turn_regs = incremental_regression(
    idq, (category.likert_turn,), beam=10, exclusions=[scale.quality],
    load='results/likert_turn_incremental_regressions_interactive'
)
likert_turn_regs

In [ ]:
likert_turn_regs_comp = incremental_regression(
    icq, (category.likert_turn,), beam=10, model='logistic', exclusions=['quality'],
    load='results/likert_turn_incremental_regressions_comparative'
)
likert_turn_regs_comp

In [ ]:
likert_turn_regs_comp = incremental_regression(
    icq, (category.likert_turn,), beam=10, model='logistic', exclusions=[scale.proactive, scale.quality],
    load='results/likert_turn_incremental_regressions_comparative_no_proactive_or_quality'
)
likert_turn_regs_comp

In [ ]:
likert_dialogue_regs = incremental_regression(
    ldq, (category.likert_dialogue,), beam=10, exclusions=['quality'],
    load='results/likert_dialogue_incremental_regressions'
)
likert_dialogue_regs

In [ ]:
likert_dialogue_regs = incremental_regression(
    idq, (category.likert_dialogue,), beam=10, exclusions=['quality'],
    load='results/likert_dialogue_incremental_regressions_interactive'
)
likert_dialogue_regs

In [ ]:
likert_dialogue_regs_comp = incremental_regression(
    icq, (category.likert_dialogue,), beam=10, model='logistic', exclusions=['quality'],
    load='results/likert_dialogue_incremental_regressions_comparative'
)
likert_dialogue_regs_comp

In [ ]:
likert_dialogue_regs_comp = incremental_regression(
    icq, (category.likert_dialogue,), beam=10, model='logistic', exclusions=[scale.engaging, scale.quality],
    load='results/likert_dialogue_incremental_regressions_comparative_no_engaging_or_quality'
)
likert_dialogue_regs_comp

In [ ]:
comparative_dialogue_regs_comp = incremental_regression(
    icq, (category.comparative,), beam=10, model='logistic', exclusions=[scale.quality],
    load='results/comparative_incremental_regressions_comparative'
)
comparative_dialogue_regs_comp

## Table for Paper

In [ ]:
def add_delta(df):
    df['R-Squared'] = (df['Adjusted R-Squared']*100)
    df['delta'] = df['R-Squared'].diff()
    df['delta'] = df['delta'].map('{:.4f}'.format)
    df['R-Squared'] = df['R-Squared'].map('{:.4f}'.format)
    df['R-Squared delta'] = df['R-Squared'] + ' (' + df['delta'] + ')'

final_behavior_regs = behavior_regs.reset_index().rename({'level_1': 'ABC-Eval'}, axis=1).drop(['level_0', 'P value of F-test'], axis=1)
add_delta(final_behavior_regs)

final_likert_turn_regs = likert_turn_regs.reset_index().rename({'level_1': 'Likert Turn'}, axis=1).drop(['level_0', 'P value of F-test'], axis=1)
add_delta(final_likert_turn_regs)

final_likert_dialogue_regs = likert_dialogue_regs.reset_index().rename({'level_1': 'Likert Dialogue'}, axis=1).drop(['level_0', 'P value of F-test'], axis=1)
add_delta(final_likert_dialogue_regs)

final_behavior_regs

In [ ]:
combined = pd.concat(
    [
        final_behavior_regs[['ABC-Eval', 'R-Squared delta']],
        final_likert_turn_regs[['Likert Turn', 'R-Squared delta']],
        final_likert_dialogue_regs[['Likert Dialogue', 'R-Squared delta']]
    ],
    axis=1)

combined.to_csv('results/paper/incremental_validity.csv', index=False)
combined